In [48]:
import numpy as np
from sklearn.preprocessing import StandardScaler
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.train.npz
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.test.npz 
 
def load_IHDP_data(training_data,testing_data,i=7):
    with open(training_data,'rb') as trf, open(testing_data,'rb') as tef:
        train_data=np.load(trf); test_data=np.load(tef)
        y=np.concatenate(   (train_data['yf'][:,i],   test_data['yf'][:,i])).astype('float32') #most GPUs only compute 32-bit floats
        t=np.concatenate(   (train_data['t'][:,i],    test_data['t'][:,i])).astype('float32')
        x=np.concatenate(   (train_data['x'][:,:,i],  test_data['x'][:,:,i]),axis=0).astype('float32')
        mu_0=np.concatenate((train_data['mu0'][:,i],  test_data['mu0'][:,i])).astype('float32')
        mu_1=np.concatenate((train_data['mu1'][:,i],  test_data['mu1'][:,i])).astype('float32')
 
        data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
        data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
        data['y']=data['y'].reshape(-1,1)
        #rescaling y between 0 and 1 often makes training of DL regressors easier
        data['y_scaler'] = StandardScaler().fit(data['y'])
        data['ys'] = data['y_scaler'].transform(data['y'])
 
    return data
 
data =load_IHDP_data(training_data='./ihdp_npci_1-100.train.npz',testing_data='./ihdp_npci_1-100.test.npz')

X,y, t = data['x'], data['y'], data['t']

File ‘ihdp_npci_1-100.train.npz’ already there; not retrieving.

File ‘ihdp_npci_1-100.test.npz’ already there; not retrieving.



In [1]:
import pandas as pd
data = pd.read_csv("data_processed_arnaud.csv")
data.set_index("Unnamed: 0", drop=True, inplace=True)
data.index.name = None
data = data.dropna()
data


from pickle import load

scaler = load(open('scaler.pkl', 'rb'))

In [5]:
covariates_cols = list(data.columns[:8])+["treatment"]
outcome = "hospital_expire_flag"
treatment = "treatment"


X = data[covariates_cols].to_numpy(dtype="float32")
t = scaler.transform(data[treatment].to_numpy(dtype="float32").reshape(-1,1))
y = data[outcome].to_numpy(dtype="float32").reshape(-1,1)


In [6]:
n_treatment = 10
t = (t*(n_treatment-1)).astype("int")

In [7]:
from tarNET import tarNET
import tensorflow as tf

normalizer_layer = tf.keras.layers.Normalization(axis=None)
normalizer_layer.adapt(X)

In [8]:
DATASET_SIZE = len(data)

batch_size = 64

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

dataset = tf.data.Dataset.zip(
    (tf.data.Dataset.from_tensor_slices((X, t)), tf.data.Dataset.from_tensor_slices(y))
).shuffle(buffer_size=DATASET_SIZE, reshuffle_each_iteration=False)#batch(64)

train_dataset = dataset.take(train_size).batch(batch_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.take(val_size).batch(batch_size)
test_dataset = test_dataset.skip(val_size)


Declare the model with correct initial bias

In [15]:
import numpy as np

neg, pos = np.bincount(np.concatenate([y for _, y in train_dataset]).reshape(-1).astype("int"))

initial_bias = tf.keras.initializers.Constant(np.log([pos/neg]))
print(f"{pos/neg*100} percent of the samples are positive")

model = tarNET(output_dim=1, n_treatments=10, normalizer_layer=normalizer_layer, scaler=scaler, output_bias=initial_bias)

20.748740100791938 percent of the samples are positive


In [23]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (neg+pos / 2.0)
weight_for_1 = (1 / pos) * (neg+pos / 2.0)

class_wts = tf.constant([weight_class_0, weight_class_1])

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.10
Weight for class 1: 5.32


In [24]:
import time
import matplotlib.pyplot as plt

loss_fn = tf.keras.losses.BinaryCrossentropy()



optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
train_acc_metric = tf.keras.metrics.BinaryAccuracy()


METRICS_logits = [
    #tf.keras.metrics.BinaryAccuracy(name="BA"),
]

METRICS_prob = [
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tf.keras.metrics.AUC(name="auc"),
    tf.keras.metrics.AUC(name="prc", curve="PR"),  # precision-recall curve
]

METRICS = METRICS_prob+METRICS_logits

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits, sample_weight=class_weight)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value, logits


@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    #val_probs = tf.round(tf.nn.sigmoid(val_logits))
    
    for m_ in METRICS:
        m_.update_state(y, val_logits)
        
    #for m_ in METRICS_prob:
        #m_.update_state(y, val_probs)
        

verbose_period=2
epochs = 50
loss_values = []
for epoch in range(epochs):
    if not(epoch %verbose_period):
        print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value, logits = train_step(x_batch_train, y_batch_train)
        if step % batch_size == 0:
            loss_values.append(loss_value)
            
        # Log every 200 batches.
        #if step % 200 == 0:
            #print(
            #    "Training loss (for one batch) at step %d: %.4f"
            #    % (step, float(loss_value))
            #)
            #print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    if not(epoch % verbose_period):
        print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)


    #metrics logits
    val_acc = {m_.name: m_.result().numpy() for m_ in METRICS_logits+METRICS_prob}
    for m_ in METRICS_prob:
        m_.reset_states()
        
    if not(epoch % verbose_period):
        print(val_acc)
    # print("Validation acc: %.4f" % (float(val_acc),))
    # print("Time taken: %.2fs" % (time.time() - start_time))
plt.plot(loss_values)


Start of epoch 0


TypeError: in user code:

    File "/var/folders/m8/sb3p17zj07s4h28sd6njtsbw0000gn/T/ipykernel_30766/3526279808.py", line 31, in train_step  *
        loss_value = loss_fn(y, logits, sample_weight=class_weight)
    File "/Users/arnaudpetit/mambaforge/lib/python3.9/site-packages/keras/losses.py", line 142, in __call__  **
        return losses_utils.compute_weighted_loss(
    File "/Users/arnaudpetit/mambaforge/lib/python3.9/site-packages/keras/utils/losses_utils.py", line 314, in compute_weighted_loss
        sample_weight = tf.convert_to_tensor(sample_weight)

    TypeError: Failed to convert elements of {0: 1.1037437005039596, 1: 5.319569743233865} to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.


In [83]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam

from IPython.display import clear_output


val_split = 0.2
batch_size = 64
verbose = 1
i = 0
tf.random.set_seed(i)
np.random.seed(i)

sgd_callbacks = [
    TerminateOnNaN(),
    EarlyStopping(monitor="val_loss", patience=40, min_delta=0.0),
    ReduceLROnPlateau(
        monitor="loss",
        factor=0.5,
        patience=5,
        verbose=verbose,
        mode="auto",
        min_delta=0.0,
        cooldown=0,
        min_lr=0,
    ),
]


adam_callbacks = [
    TerminateOnNaN(),
    EarlyStopping(monitor="val_loss", patience=20, min_delta=0.0),
]

# optimzier hyperparameters
sgd_lr = 1e-5
momentum = 0.9

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)


model.compile(
    optimizer=Adam(),  # SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
    loss=loss,
    metrics=tf.keras.metrics.BinaryAccuracy(),
)

history = model.fit(
    train_dataset,
    #x=[X,t],
    #y=y,
    #shuffle=True,
    callbacks=adam_callbacks,
    #validation_split=val_split,
    epochs=10,
    batch_size=batch_size,
    verbose=verbose,
)

clear_output()

acc = history.history["binary_accuracy"]
val_acc = history.history["val_binary_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

import matplotlib.pyplot as plt

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

plt.show()

Epoch 1/10
8386/8386 [==============================] - 51s 6ms/step - loss: 0.5082 - binary_accuracy: 0.8356
Epoch 2/10
8386/8386 [==============================] - 52s 6ms/step - loss: 0.4707 - binary_accuracy: 0.8338
Epoch 3/10
8386/8386 [==============================] - 66s 8ms/step - loss: 0.4524 - binary_accuracy: 0.8339
Epoch 4/10
8386/8386 [==============================] - 60s 7ms/step - loss: 0.4609 - binary_accuracy: 0.8337
Epoch 5/10
 653/8386 [=>............................] - ETA: 1:03 - loss: 0.5303 - binary_accuracy: 0.8086

KeyboardInterrupt: 